In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from PIL import Image
import random

import os
import json
import cv2

import torch
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import datasets, transforms

from transformers import TrOCRProcessor, VisionEncoderDecoderModel, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import albumentations as A



Завиксируем всю случайность!

In [2]:
seed = 23
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.determenistic = True

Соберем наш датасет из полученных файлов

In [3]:
basketball_df = pd.read_csv('for_train\\train_bascetball.csv')
streetball_df = pd.read_csv('for_train\\train_streetball.csv')
volleyball_df = pd.read_csv('for_train\\train_volleyball.csv')

res_df = pd.concat([basketball_df, streetball_df, volleyball_df], axis=0)
res_df.shape

(150630, 3)

Разобьем датасет на train test и eval части

In [4]:
diff_df, test_df, = train_test_split(res_df, test_size=0.1, shuffle=True)
train_df, eval_df = train_test_split(diff_df, test_size=0.001)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [5]:
img_size = 386

train_augs = A.Compose([A.Resize(img_size, img_size),
                A.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				A.CLAHE(clip_limit=(1,4), p=1),
                #A.imgaug.transforms.IAASharpen(alpha=(0.2, 0.3), lightness=(0.5, 0.7), p=1),
                #A.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),				
               ])

test_augs = A.Compose([A.Resize(img_size, img_size)])

In [6]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=3, transforms = None):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
   
        image = Image.open(file_name).convert("RGB")
        #image = cv2.imread(file_name)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       
        if self.transforms:
                image = self.transforms(image=np.array(image))['image']
        else:
            image = image.resize((img_size, img_size))                    
        
        #image = image.resize((img_size, img_size))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

Создадим наши датасеты

In [7]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor,
                           transforms=test_augs)
test_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor,
                           transforms=test_augs)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=eval_df,
                           processor=processor,
                           transforms=test_augs)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
print("Number of training examples:", len(train_dataset))
print("Number of testing examples:", len(test_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 135431
Number of testing examples: 15063
Number of validation examples: 136


Загрузим предъобученный трансформер

In [9]:
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
#model = VisionEncoderDecoderModel.from_pretrained("model")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained('model')
model.to(device)
print(device)

cuda


Сконфигурируем нашу модель

In [11]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 3
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Сконфигурируем цикл обучения

In [12]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=40,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="G:\\models1",
    logging_steps=50,
    save_steps=10000,
    eval_steps=50,
)

Оопределим метрику

In [13]:
import evaluate

cer_metric = evaluate.load("cer")
acc_metric = evaluate.load("accuracy")

In [14]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions    
    #print(pred_ids)
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)    

    label_int = [int(x) for x in label_str]
    x = [] 
    for j in pred_str:       
        if j.isdigit():
            x.append(int(j))
        else:
            x.append(1000)
    #print(label_int)
    #print(pred_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    acc = acc_metric.compute(predictions=x, references=label_int)
    #print(acc)
    return {"cer": cer,
            "accuracy": acc}


Переопределим оптимизатор

In [15]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),lr=0.00001)

Запустим цикл обучения

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

model.save_pretrained('model_next')

c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1805760 [00:00<?, ?it/s]

{'loss': 0.0935, 'learning_rate': 4.999864323055113e-05, 'epoch': 0.0}


  0%|          | 0/46 [00:00<?, ?it/s]

{'eval_loss': 7.655216217041016, 'eval_cer': 1.0, 'eval_accuracy': {'accuracy': 0.0}, 'eval_runtime': 18.491, 'eval_samples_per_second': 7.355, 'eval_steps_per_second': 2.488, 'epoch': 0.0}
{'loss': 0.5333, 'learning_rate': 4.999736952861953e-05, 'epoch': 0.0}


  0%|          | 0/46 [00:00<?, ?it/s]

{'eval_loss': 7.905254364013672, 'eval_cer': 1.0, 'eval_accuracy': {'accuracy': 0.0}, 'eval_runtime': 18.513, 'eval_samples_per_second': 7.346, 'eval_steps_per_second': 2.485, 'epoch': 0.0}


KeyboardInterrupt: 

In [ ]:
url = "crops\\4\\basketball_3383example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Номер на футболке: 4


Определим eval_dataloader и Загрузим обученную модель

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32)
model = VisionEncoderDecoderModel.from_pretrained('G:\\models1\\checkpoint-40000')

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

torch.cuda.empty_cache()
# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
diff_acc = []
acc = []
i = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
    
        target_text = processor.batch_decode(batch['labels'], skip_special_tokens=True)
        target_text = [int(x) for x in target_text]
           
        pixel_values = batch['pixel_values']
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=4)       
       
        # Make a prediction
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

        x = [] 
        for j in generated_text:       
            if j.isdigit():
                x.append(int(j))
            else:
                x.append(1000)        
      
        bach_acc = accuracy_score(target_text, x)
        # Save the true and predicted labels
        diff_acc.append(bach_acc)  
        acc.append(bach_acc)

        if (i % 50  == 0) & (i > 2):
            accuracy = np.mean(diff_acc)
            print("Accuracy:", accuracy) 
        i += 1

    print(f"Total accuracy: {np.mean(acc)}")    

  0%|          | 0/942 [00:00<?, ?it/s]

Accuracy: 0.8768382352941176
Accuracy: 0.880569306930693
Accuracy: 0.8849337748344371
Accuracy: 0.8851057213930348
Accuracy: 0.8813496015936255
Accuracy: 0.8822674418604651
Accuracy: 0.8839921652421653
Accuracy: 0.8827150872817955
Accuracy: 0.8817904656319291
Accuracy: 0.8832959081836327
Accuracy: 0.8844714156079855
Accuracy: 0.8843074043261231
Accuracy: 0.8849366359447005
Accuracy: 0.8859218972895863
Accuracy: 0.8852363515312917
Accuracy: 0.8845583645443196
Accuracy: 0.8843639835487661
Accuracy: 0.884225860155383
Total accuracy: 0.8842887473460722


Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
y_true = []
y_pred = []
xx = []
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), nrows=2):
        url = test_df['file_name'][i]
        xx.append(url)
        target_text = int(test_df['text'][i])
        image = Image.open(url).convert("RGB")   

        pixel_values = processor(image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=3)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        #pixel_values = batch['pixel_values']
        #generated_ids = model.generate(pixel_values.cuda())       

        # Make a prediction
        #generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
        x = []        
        if generated_text.isdigit():
            
            x = int(generated_text)
        else:
            x = 1000    
        
        # Save the true and predicted labels
        y_true.append(target_text)
        y_pred.append(x)

        if (i % 400  == 0) & (i > 2):
            accuracy = accuracy_score(y_true, y_pred)
            print("Accuracy:", accuracy) 

print("Accuracy:", accuracy)                   